In [89]:
import scipy.io
import numpy as np
mat = scipy.io.loadmat('mnist_all.mat')

In [90]:

def inner_product(X, Y, W_mu, p):
    if X.shape[0] != Y.shape[0]:
        raise Exception('Mismatched shape: \n Shape of X = ' + str(X.shape) + '\n' + 'Shape of Y = ' + str(Y.shape))
    
    if(p==2):
        return np.dot(X, Y)

    s = 0

    for i in range(X.shape[0]):
        n_ij = np.power(np.abs(W_mu[i]), p-2)
        s += n_ij*X[i]*Y[i]
    return s

In [91]:
hidden = 200
inp = 784

lr = 1e-1
delta = 0.4

list(mat.keys())
mat['train0']
ex = (mat['train0'][0]/255.0)

W = np.random.normal(0.0, 1.0, (inp, hidden))



In [92]:
Q = np.zeros((hidden, 1))
for j in range(hidden):
    Q[j] = inner_product(ex, W[:,j], W[:,j], 2)


def activation(Q):
    Q_sort = Q.argsort(axis=0)
    new_Q = np.zeros((hidden, 1))
    new_Q[Q_sort[-1][0]] = 1.0
    new_Q[Q_sort[-2][0]] = -delta
    return new_Q


In [93]:
Q_sort = Q.argsort(axis=0)

In [94]:
Q_sort.shape
Q[Q_sort[-1][0]]

array([31.6543129])

In [95]:
for k in range(200):
    Q = np.zeros((hidden, 1))
    for j in range(hidden):
        Q[j] = inner_product(ex, W[:,j], W[:,j], 2)

    dW = np.zeros_like(W)
    for j in range(hidden):
        for i in range(inp):
            dW_ij = activation(Q)[j]*(ex[i] - inner_product(ex, W[:,j], W[:,j], 2)*W[i, j])
            dW[i, j] = dW_ij
    print(dW.max())

    W += lr*dW/np.argmax(dW)
    print(inner_product(W[:, j], W[:,j], W[:,j], 2))

97.55442102074703
796.6029480406689
97.54843370545173
796.6029480406689
97.542446950285
796.6029480406689


KeyboardInterrupt: 

In [ ]:
inner_product(W[:, j], W[:,j], W[:,j], 2)

737.3163719321303